In [1]:
from sklearn.datasets import load_iris
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [2]:
m = nn.Linear(20,30)

In [3]:
input = torch.ones(128, 20)

In [4]:
output = m(input)

In [5]:
output

tensor([[-1.6946, -0.0449, -0.4358,  ...,  0.3827,  0.9301, -0.0555],
        [-1.6946, -0.0449, -0.4358,  ...,  0.3827,  0.9301, -0.0555],
        [-1.6946, -0.0449, -0.4358,  ...,  0.3827,  0.9301, -0.0555],
        ...,
        [-1.6946, -0.0449, -0.4358,  ...,  0.3827,  0.9301, -0.0555],
        [-1.6946, -0.0449, -0.4358,  ...,  0.3827,  0.9301, -0.0555],
        [-1.6946, -0.0449, -0.4358,  ...,  0.3827,  0.9301, -0.0555]],
       grad_fn=<AddmmBackward>)

In [6]:
a = np.arange(10).astype(np.float32)
a_tensor = torch.from_numpy(a).clone()
m = F.log_softmax(a_tensor)
m



<ipython-input-6-bc57ef3bb176>:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  m = F.log_softmax(a_tensor)


tensor([-9.4586, -8.4586, -7.4586, -6.4586, -5.4586, -4.4586, -3.4586, -2.4586,
        -1.4586, -0.4586])

In [7]:
iris = load_iris()

class Net(nn.Module):
    ## 個々で
    def __init__(self):
        ## nn.Moduleを継承するよってことで。
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 3)
    def forward(self, x):
        x = self.fc1(x)
        ## 最後にlog softmax(ココは普通にF.softmax()に変えても処理は回ります。)で出力
        return F.log_softmax(x, dim = 1)

In [8]:
model = Net()
train_X = torch.Tensor(iris.data)       ## torch.Tensor型: Pytorch版のnumpy形式
train_y = torch.LongTensor(iris.target)


In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.02)
## エポック数(何回学習させるか)
epochs = 1000
## 後の可視化用に取っておく
loss_log, accuracy_log = [], []
w = []

data, target = Variable(train_X), Variable(train_y)
for epoch in range(epochs):
    ## pytorchでは勾配を蓄積する仕組みなので更新前に初期化しておきます
    optimizer.zero_grad()
    ## feed forward(つまり予測させます)
    output = model(data)
    ## 予実差からの誤差を決めます。nll_lossは、Negative Log Likelihood(負の対数尤度)ですね。
    loss = F.nll_loss(output, target)

    ## Back Propagation
    loss.backward()
    w.append(list(model.parameters())[0])
    # print(list(model.parameters())[0].grad)
    ## 各層内のパラメータの更新
    optimizer.step()


    ## 正解率の計算
    prediction = output.data.max(1)[1]
    accuracy = prediction.eq(target.data).sum().numpy() / len(iris.data)

    ## 記録を取っておきます
    loss_log.append(loss.item())
    accuracy_log.append(accuracy)

    if epoch % 100 == 0:
        print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(epoch, loss.data.item(), accuracy))

Train Step: 0	Loss: 2.297	Accuracy: 0.327
Train Step: 100	Loss: 0.559	Accuracy: 0.893
Train Step: 200	Loss: 0.457	Accuracy: 0.920
Train Step: 300	Loss: 0.406	Accuracy: 0.953
Train Step: 400	Loss: 0.372	Accuracy: 0.967
Train Step: 500	Loss: 0.346	Accuracy: 0.967
Train Step: 600	Loss: 0.324	Accuracy: 0.967
Train Step: 700	Loss: 0.306	Accuracy: 0.973
Train Step: 800	Loss: 0.290	Accuracy: 0.973
Train Step: 900	Loss: 0.276	Accuracy: 0.973


In [10]:
a = list(model.parameters())



In [11]:
a

[Parameter containing:
 tensor([[ 0.3532,  0.7583, -1.3931, -0.6541],
         [ 0.1787, -0.6736,  0.2229, -0.0440],
         [-0.9763, -0.9781,  1.7044,  1.2810]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2214,  0.5510, -0.7020], requires_grad=True)]